<io>
Word2vec 은 원핫인코딩을 기반으로, 단어간 유사도를 파악할 수 있도록 하는 벡터화 방법이다.<br>
이를 기반으로, 10-K 에서 사업설명 부분에서 각 기업의 사업설명, 전략적 자원 식별을 바탕으로 비슷한 위치에서 등장하는 단어가 비슷한 의미라는 것을 이용하여 <br>
전체 10-K 단어집을 완성시켜보고자 한다. 이를 토대로, 이후 여러 단어집 형성에도 도움이 될 것으로 예상된다. <br>


### CBOW (Continous Bag of Words))
<oi>
주변의 있는 단어 기준으로 윈도우를 형성하여 중심 단어의 벡터를 예측/학습 하는 방식

### 먼저 한개 문서 (사업설명 부분)에 대해서 연습을 해 보고, 전체 문서로 단어집을 형성해보고자 한다.

In [ ]:
import os
import re
import nltk
from nltk.stem.porter import PorterStemmer

dir = "/media/siro8458/Data/본체/자산평가/sentence/"
files = os.listdir(dir)
i = 0

with open(dir+files[i], 'r', encoding='utf-8') as file:
    file_content = file.read()
file_name = files[i]
# 연도 추출 - 문자 사이의 4자리 숫자임
year_match = re.search(r'\b\d{4}\b', file_name)
year = year_match.group() if year_match else None
# 회사 이름 추출 (연도 바로 앞의 - 이전이 전부 회사이름임)
if year:
    firmname = file_name.split(year)[0].rstrip('-')
else:
    firmname = None
    
file_content = file_content.split('\n') #\n 형태로 메모장에 기록되어 있으므로 나누어줌
tokenList = [sent.split() for sent in file_content]

stemmer = PorterStemmer()

processed_stemmed_list = []
for sentence in tokenList:
    processed_sentence = []
    for word in sentence:
        stemmed_word = stemmer.stem(word)
        processed_sentence.append(stemmed_word)
    processed_stemmed_list.append(processed_sentence)

In [26]:
processed_stemmed_list[:3] #위에서 부터 3개만 간단히 봄

[['miez', 'inc'],
 ['includ',
  'it',
  'wholli',
  'own',
  'subsidiari',
  'is',
  'a',
  'lead',
  'specialti',
  'retail',
  'of',
  'apparel',
  'footwear',
  'accessori',
  'and',
  'hardgood',
  'for',
  'young',
  'men',
  'and',
  'women',
  'who',
  'want',
  'to',
  'express',
  'their',
  'individu',
  'through',
  'the',
  'fashion',
  'music',
  'art',
  'and',
  'cultur',
  'of',
  'action',
  'sport',
  'streetwear',
  'and',
  'other',
  'uniqu',
  'lifestyl'],
 ['zumiez', 'inc']]

In [15]:
file_name

'Zumiez Inc-2015.0.txt'

<io>
Word2vec 학습 <br>
window 옵션이 윈도우 창을 지정해주며, min_count 옵션은 최소 빈도 단어 수로, 이 이하의 빈도수의 경우 학습에 사용안됨 <br>
sg 옵션의 경우, 0일 경우 CBOW, 1일 경우 Skip-Gram 으로 현재는 0을 지정하여 학습을 진행하여 CBOW 의 결과를 보고자 한다.


In [31]:
from gensim.models import Word2Vec
cbow_model1 = Word2Vec(sentences=processed_stemmed_list, window=5, min_count=3,  sg=0)

In [21]:
cbow_model1.wv.most_similar("product") #product와 유사한 단어를 추출해본 결과, stop word에 속하는 단어가 많은 듯 하다.

[('the', 0.999642550945282),
 ('our', 0.9996387362480164),
 ('other', 0.9996066093444824),
 ('and', 0.9996013045310974),
 ('to', 0.9995980858802795),
 ('in', 0.999582052230835),
 ('of', 0.9995809197425842),
 ('a', 0.9995554685592651),
 ('store', 0.9995415210723877),
 ('or', 0.9995406270027161)]

<io>
현재 stopword에 속할마한 the, and 등이 너무 많이 잡히는 탓에 한번 제거를 한 결과로 토대로 다시 해 보고자 한다.

In [27]:
from nltk.corpus import stopwords
#처음 켰을때 한번만 실행
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

processed_stemmed_list2 = []
for sentence in tokenList:
    processed_sentence = []
    for word in sentence:
        if not (word in stop_words or re.match(r'\d+', word)):  #stop_word나 숫자를 제거, 즉 아닌경우
            # Porter Stemmer 적용
            stemmed_word = stemmer.stem(word)
            processed_sentence.append(stemmed_word)
    processed_stemmed_list2.append(processed_sentence)


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/siro8458/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [28]:
processed_stemmed_list2[:2]

[['miez', 'inc'],
 ['includ',
  'wholli',
  'own',
  'subsidiari',
  'lead',
  'specialti',
  'retail',
  'apparel',
  'footwear',
  'accessori',
  'hardgood',
  'young',
  'men',
  'women',
  'want',
  'express',
  'individu',
  'fashion',
  'music',
  'art',
  'cultur',
  'action',
  'sport',
  'streetwear',
  'uniqu',
  'lifestyl']]

In [30]:
cbow_model2 = Word2Vec(sentences=processed_stemmed_list2, window=5, min_count=3,  sg=0)

In [32]:
cbow_model1.wv.most_similar("product") 

[('the', 0.9996445178985596),
 ('our', 0.9996435642242432),
 ('other', 0.9996076822280884),
 ('and', 0.9996013045310974),
 ('to', 0.9995996952056885),
 ('in', 0.9995824098587036),
 ('of', 0.9995821118354797),
 ('a', 0.9995564222335815),
 ('or', 0.9995422959327698),
 ('store', 0.999542236328125)]

In [33]:
cbow_model2.wv.most_similar("product") 

[('control', 0.950051486492157),
 ('us', 0.9492754340171814),
 ('could', 0.9425510168075562),
 ('store', 0.9416999816894531),
 ('oper', 0.9382789731025696),
 ('cost', 0.9377127885818481),
 ('busi', 0.9341433048248291),
 ('merchandis', 0.9321338534355164),
 ('may', 0.9300434589385986),
 ('risk', 0.927400529384613)]

In [38]:
words1 = cbow_model1.wv.most_similar("product", topn=1000)  #topn 옵션을 넣으면 상위 몇개 유사 단어를 볼지를 확인 가능, default는 10
next(((word, similarity) for word, similarity in words1 if word == "merchandis"), None)



('merchandis', 0.9993808269500732)

<io>
    product 와 유사한 단어로 상위 1000개 단어집을 형성하는 경우, stopwords를 하지 않더라도 똑같이 merchandis가 출력이 되는 것으로 보인다.<br>
    stopword 여부는 의미가 없을 수 있지만, 현재 한개 문장에는 단어 수 등의 표본이 적을 수 있기에, 더 많은 표본을 넣고, stopword 와 단어집을 한번 비교를 해보도록 하자

In [1]:
import os
import re
import nltk
from nltk.stem.porter import PorterStemmer

from nltk.corpus import stopwords
#처음 켰을때 한번만 실행
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

dir = "/media/siro8458/Data/본체/자산평가/sentence/"
files = os.listdir(dir)

processed_stemmed_list1 = []
processed_stemmed_list2 = []


for i in range(0, len(files)):
    with open(dir+files[i], 'r', encoding='utf-8') as file:
        file_content = file.read()
    file_name = files[i]
    # 연도 추출 - 문자 사이의 4자리 숫자임
    year_match = re.search(r'\b\d{4}\b', file_name)
    year = year_match.group() if year_match else None
    # 회사 이름 추출 (연도 바로 앞의 - 이전이 전부 회사이름임)
    if year:
        firmname = file_name.split(year)[0].rstrip('-')
    else:
        firmname = None
        
    file_content = file_content.split('\n') #\n 형태로 메모장에 기록되어 있으므로 나누어줌
    tokenList = [sent.split() for sent in file_content]
    
    stemmer = PorterStemmer()
    
    
    for sentence in tokenList:
        processed_sentence1 = []
        processed_sentence2 = []
        for word in sentence:
            stemmed_word = stemmer.stem(word)
            processed_sentence1.append(stemmed_word)
        processed_stemmed_list1.append(processed_sentence1)

        for word in sentence:
            if not (word in stop_words or re.match(r'\d+', word)):  #stop_word나 숫자를 제거, 즉 아닌경우
                # Porter Stemmer 적용
                stemmed_word = stemmer.stem(word)
                processed_sentence2.append(stemmed_word)
        processed_stemmed_list2.append(processed_sentence2)

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/siro8458/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
processed_stemmed_list2[:5]

[['miez', 'inc'],
 ['includ',
  'wholli',
  'own',
  'subsidiari',
  'lead',
  'specialti',
  'retail',
  'apparel',
  'footwear',
  'accessori',
  'hardgood',
  'young',
  'men',
  'women',
  'want',
  'express',
  'individu',
  'fashion',
  'music',
  'art',
  'cultur',
  'action',
  'sport',
  'streetwear',
  'uniqu',
  'lifestyl'],
 ['zumiez', 'inc'],
 ['form', 'august', 'washington', 'state', 'corpor'],
 ['januari', 'oper', 'store', 'unit', 'state', 'us', 'canada', 'europ']]

In [ ]:
cbow_model1 = Word2Vec(sentences=processed_stemmed_list1, window=5, min_count=3,  sg=0)
cbow_model2 = Word2Vec(sentences=processed_stemmed_list2, window=5, min_count=3,  sg=0)

In [15]:
similar1 = cbow_model1.wv.most_similar("product", topn=1000)
similar2 = cbow_model2.wv.most_similar("product", topn=1000)

f_words1= [word for word, similarity in similar1 if similarity > 0.3]
f_words2= [word for word, similarity in similar2 if similarity > 0.3]



In [19]:
import pandas as pd

d1 = pd.DataFrame(f_words1, columns = (['add_stop']))
d2 = pd.DataFrame(f_words2, columns = (['not_stop']))
pd.concat([d1, d2], axis=1) #stop_words가 없는 경우 더 단어가 많이 뽑히는 것처럼 보임

,add_stop,not_stop
0,drug,manufactur
1,merchandis,drug
2,vaccin,merchandis
3,manufactur,vaccin
4,solut,microorgan
...,...,...
788,NaN,clickpad
789,NaN,suppositori
790,NaN,radiometr
791,NaN,polycarbon


<oi>
 각 회사의 대표적인 상품들이 들어와 있는 것으로 추정되며, CBOW 방식을 이용하였을 경우 어느정도의 각 회사의 상품으로 추정되는 문구를 찾는 듯 하다.<br>
 또한 여러 회사들에서 하는 것과 마찬가지로 similarity가 0.3이상인 결과도 다음과 같고, skip-gram까지 합쳐서 risk와 supply 관련 단어를 통하여 각 회사별 supply_risk 추정할 계획

In [4]:
import time
from gensim.models import Word2Vec

start_time = time.time()
cbow_model1 = Word2Vec(sentences=processed_stemmed_list1, window=5, min_count=3,  sg=0)
end_time = time.time()
total_time = (end_time - start_time)
print("CBOW_add_stop : ", total_time)

start_time = time.time()
cbow_model2 = Word2Vec(sentences=processed_stemmed_list2, window=5, min_count=3,  sg=0) 
end_time = time.time()
total_time = (end_time - start_time)
print("CBOW_not_stop : ", total_time)

start_time = time.time()
cbow_model3 = Word2Vec(sentences=processed_stemmed_list1, window=5, min_count=3,  sg=1) #skip_words
end_time = time.time()
total_time = (end_time - start_time)
print("SKIP_add_stop : ", total_time)

start_time = time.time()
cbow_model4 = Word2Vec(sentences=processed_stemmed_list2, window=5, min_count=3,  sg=1) #skip_words_except stop words
end_time = time.time()
total_time = (end_time - start_time)
print("SKIP_not_stop : ", total_time)

CBOW_add_stop :  360.00419068336487
CBOW_not_stop :  265.51319313049316
SKIP_add_stop :  1200.4223911762238
SKIP_not_stop :  833.9526751041412


In [21]:
print("supply : ", stemmer.stem("supply"))
print("risk : ", stemmer.stem("risk"))

supply :  suppli
risk :  risk


In [22]:
supply1 = cbow_model1.wv.most_similar("suppli", topn=1000)
supply2 = cbow_model2.wv.most_similar("suppli", topn=1000)
supply3 = cbow_model3.wv.most_similar("suppli", topn=1000)
supply4 = cbow_model4.wv.most_similar("suppli", topn=1000)

s_words1= [word for word, similarity in supply1 if similarity > 0.3]
s_words2= [word for word, similarity in supply2 if similarity > 0.3]
s_words3= [word for word, similarity in supply3 if similarity > 0.3]
s_words4= [word for word, similarity in supply4 if similarity > 0.3]

s1 = pd.DataFrame(f_words1, columns = (['add_stop_CBOW']))
s2 = pd.DataFrame(f_words2, columns = (['not_stop_CBOW']))
s3 = pd.DataFrame(f_words1, columns = (['add_stop_SKIP']))
s4 = pd.DataFrame(f_words2, columns = (['not_stop_SKIP']))
supply = pd.concat([s1,s2,s3,s4], axis=1) 

risk1 = cbow_model1.wv.most_similar("risk", topn=1000)
risk2 = cbow_model2.wv.most_similar("risk", topn=1000)
risk3 = cbow_model3.wv.most_similar("risk", topn=1000)
risk4 = cbow_model4.wv.most_similar("risk", topn=1000)

r_words1= [word for word, similarity in risk1 if similarity > 0.3]
r_words2= [word for word, similarity in risk2 if similarity > 0.3]
r_words3= [word for word, similarity in risk3 if similarity > 0.3]
r_words4= [word for word, similarity in risk4 if similarity > 0.3]

r1 = pd.DataFrame(r_words1, columns = (['add_stop_CBOW']))
r2 = pd.DataFrame(r_words2, columns = (['not_stop_CBOW']))
r3 = pd.DataFrame(r_words1, columns = (['add_stop_SKIP']))
r4 = pd.DataFrame(r_words2, columns = (['not_stop_SKIP']))
risk = pd.concat([r1,r2,r3,r4], axis=1) 

In [23]:
supply

,add_stop_CBOW,not_stop_CBOW,add_stop_SKIP,not_stop_SKIP
0,drug,manufactur,drug,manufactur
1,merchandis,drug,merchandis,drug
2,vaccin,merchandis,vaccin,merchandis
3,manufactur,vaccin,manufactur,vaccin
4,solut,microorgan,solut,microorgan
...,...,...,...,...
788,NaN,clickpad,NaN,clickpad
789,NaN,suppositori,NaN,suppositori
790,NaN,radiometr,NaN,radiometr
791,NaN,polycarbon,NaN,polycarbon


In [24]:
risk

,add_stop_CBOW,not_stop_CBOW,add_stop_SKIP,not_stop_SKIP
0,exposur,exposur,exposur,exposur
1,riskth,riskth,riskth,riskth
2,riskw,riskw,riskw,riskw
3,11risk,uncertainti,11risk,uncertainti
4,22risk,risksw,22risk,risksw
...,...,...,...,...
74,riskier,software2014bannomobil,riskier,software2014bannomobil
75,hurdl,gaap_residentialmortgagememb,hurdl,gaap_residentialmortgagememb
76,gleam,riskier,gleam,riskier
77,company4t,NaN,company4t,NaN


In [25]:
supply.to_csv('supply.csv',index=False)
risk.to_csv('risk.csv',index=False)

<io>
skip 과 cbow 모두 비슷하게 뜨는 듯 하다. 또한, supply의 경우 이전 product와 유사한 단어가 뜨는 듯 하며, 단순 수출이 아닌 상품에 대한 ㄹ과를 보이는 듯 하다<br>
    구성된 단어집을 바탕으로 각 기업의 전체 token 에서 supply risk의 비율을 계산해 보고자 한다.

In [54]:
dir = "/media/siro8458/Data/본체/자산평가/sentence/"
files = os.listdir(dir)

processed_stemmed_list1 = []
supply_risk = {}

add_stop_CBOW_s = supply['add_stop_CBOW'].dropna().tolist()
not_stop_CBOW_s = supply['not_stop_CBOW'].dropna().tolist()
add_stop_SKIP_s = supply['add_stop_SKIP'].dropna().tolist()
not_stop_SKIP_s = supply['not_stop_SKIP'].dropna().tolist()

add_stop_CBOW_r = risk['add_stop_CBOW'].dropna().tolist()
not_stop_CBOW_r = risk['not_stop_CBOW'].dropna().tolist()
add_stop_SKIP_r = risk['add_stop_SKIP'].dropna().tolist()
not_stop_SKIP_r = risk['not_stop_SKIP'].dropna().tolist()


for i in range(0,len(files)):
    with open(dir+files[i], 'r', encoding='utf-8') as file:
        file_content = file.read()
    file_name = files[i]
    # 연도 추출 - 문자 사이의 4자리 숫자임
    year_match = re.search(r'\b\d{4}\b', file_name)
    year = year_match.group() if year_match else None
    # 회사 이름 추출 (연도 바로 앞의 - 이전이 전부 회사이름임)
    if year:
        firmname = file_name.split(year)[0].rstrip('-')
    else:
        firmname = None

    key = f"{firmname}-{year}"
    file_content = file_content.split('\n') #\n 형태로 메모장에 기록되어 있으므로 나누어줌
    tokenList = [sent.split() for sent in file_content]
       
    
    sr_count1 = 0
    sr_count2 = 0
    sr_count3 = 0
    sr_count4 = 0 #각 문서별 전체 suupply risk 수 카운트 위해서    
    for sentence in tokenList:
        s_count1 = 0 #각 문장별 supply랑 risk 여부 확인하기 위해서
        r_count1 = 0
        s_count2 = 0
        r_count2 = 0
        s_count3 = 0
        r_count3 = 0
        s_count4 = 0
        r_count4 = 0
        for word in sentence:
            #STOP WORD 포함된 CBOW
            if word in add_stop_CBOW_s:
                s_count1+=1 #그 문장에 suupply 단어가 있으면 +1
            if word in add_stop_CBOW_r:
                r_count1+=1 #그 문장에 risk 단어가 있으면 +1
            #STOP WORD 제거된 CBOW
            if word in not_stop_CBOW_s:
                s_count2+=1 #그 문장에 suupply 단어가 있으면 +1
            if word in not_stop_CBOW_r:
                r_count2+=1 #그 문장에 risk 단어가 있으면 +1
            #STOP WORD 포함된 SKIP
            if word in add_stop_SKIP_s:
                s_count3+=1 #그 문장에 suupply 단어가 있으면 +1
            if word in add_stop_SKIP_r:
                r_count3+=1 #그 문장에 risk 단어가 있으면 +1
            #STOP WORD 제거된 SKIP
            if word in not_stop_SKIP_s:
                s_count4+=1 #그 문장에 suupply 단어가 있으면 +1
            if word in not_stop_SKIP_r:
                r_count4+=1 #그 문장에 risk 단어가 있으면 +1
        if s_count1*r_count1 > 0: #문장안에서 반복문이 끝났으니, 둘 곱이 0이 아니면 supply와 risk 단어가 모두 나타난다는 뜻
            sr_count1+=1 #supply risk +1 해줌
        if s_count2*r_count2 > 0: 
            sr_count2+=1 
        if s_count3*r_count3 > 0: 
            sr_count3+=1 
        if s_count4*r_count4 > 0: 
            sr_count4+=1 
    supply_risk[key] = {'sr_add_stop_CBOW' : sr_count1/len(tokenList), 'sr_not_stop_CBOW' : sr_count2/len(tokenList),
                       'sr_add_stop_SKIP' : sr_count3/len(tokenList), 'sr_not_stop_SKIP' : sr_count4/len(tokenList),}

In [55]:
import pickle

with open('supply_risk.pkl', 'wb') as f:
    pickle.dump(supply_risk, f)
    
with open('supply_risk.pkl', 'rb') as f:
    supply_risk = pickle.load(f)

### 기존의 CAR 등과 같이 모아놓은거랑 합쳐서 분석을 진행해봄

In [68]:
ddd = pd.read_csv('CAR_control2.csv', encoding='cp949', index_col=0)
ss = pd.read_csv('gram_sample_CIK_filed.csv', encoding='cp949')

sr_add_stop_CBOW = []
sr_not_stop_CBOW = []
sr_add_stop_SKIP = []
sr_not_stop_SKIP = []

for i in range(0, len(ddd)):
    key = ss.loc[i, 'name']+'-'+str(ss.loc[i, 'year'])
    sr_add_stop_CBOW.append(supply_risk[key]['sr_add_stop_CBOW'])
    sr_not_stop_CBOW.append(supply_risk[key]['sr_not_stop_CBOW'])
    sr_add_stop_SKIP.append(supply_risk[key]['sr_add_stop_SKIP'])
    sr_not_stop_SKIP.append(supply_risk[key]['sr_not_stop_SKIP'])

sr_add_stop_CBOW = pd.DataFrame(sr_add_stop_CBOW, columns=['sr_add_stop_CBOW'])
sr_not_stop_CBOW = pd.DataFrame(sr_not_stop_CBOW, columns=['sr_not_stop_CBOW'])
sr_add_stop_SKIP = pd.DataFrame(sr_add_stop_SKIP, columns=['sr_add_stop_SKIP'])
sr_not_stop_SKIP = pd.DataFrame(sr_not_stop_SKIP, columns=['sr_not_stop_SKIP'])

ddd = pd.concat([ddd, sr_add_stop_CBOW, sr_not_stop_CBOW, sr_add_stop_SKIP, sr_not_stop_SKIP], axis=1)
ddd.to_csv('CAR_control_supply_risk.csv', encoding='cp949')

In [75]:
import statsmodels.formula.api as smf
from statsmodels.iolib.summary2 import summary_col
import pandas as pd

ddd = pd.read_csv('CAR_control_supply_risk.csv', encoding='cp949', index_col=0)

X = ['sr_add_stop_CBOW','sr_not_stop_CBOW', 'sr_add_stop_SKIP', 'sr_not_stop_SKIP']

models = []

for x in X:
    if '.' in x: #0.001과 같은 경우 변수명이 안읽혀서 변수명을 새로 하기보단 방법을 찾아보니 {} 형태로 감싸기로 ㅎㅁ
        formula = "CAR ~ sale + lnmkv + bperhare + Q('{}')".format(x)
    else:
        formula = 'CAR ~ sale + lnmkv + bperhare + {}'.format(x)
    model = smf.ols(formula, data=ddd).fit()
    models.append(model)
          
dfoutput = summary_col(models, stars=True, model_names=X)
print(dfoutput)


                 sr_add_stop_CBOW sr_not_stop_CBOW sr_add_stop_SKIP sr_not_stop_SKIP
------------------------------------------------------------------------------------
Intercept        0.0076***        0.0077***        0.0076***        0.0077***       
                 (0.0028)         (0.0028)         (0.0028)         (0.0028)        
sale             0.0000           0.0000           0.0000           0.0000          
                 (0.0000)         (0.0000)         (0.0000)         (0.0000)        
lnmkv            -0.0012***       -0.0012***       -0.0012***       -0.0012***      
                 (0.0004)         (0.0004)         (0.0004)         (0.0004)        
bperhare         0.0000***        0.0000***        0.0000***        0.0000***       
                 (0.0000)         (0.0000)         (0.0000)         (0.0000)        
sr_add_stop_CBOW -0.0542                                                            
                 (0.1495)                                       

<io>
 Supply Risk에 대해서 이전에 구한 공시일 직후의 CAR 1 의 변화에는 유의성이 나타나지 않았으나, 부호는 음의 방향을 나타내었다. <br>
 이는 공급망 자체가 장기적인 효과를 추정하기도 하고, 선행 연구들과 같이 volatility가 Cash ratio나 Inventory 를 기반으로 추정을 한다면 유사한 결과가 나타날 듯 하다 <br>
 음의 방향이라는 것 자체는 결국 supply risk 역시 위험리스크 를 나타내기에 높으면 높을수록, CAR 등에 음의 영향을 끼친다는 직관과는 일치한다. <br>
 CBOW와 SKIP의 결과 자체는 결국 단어집 구성에는 차이가 없고, 성능 차이 즉 속도의 차이만을 보였다는 것을 확인 가능하다. (약 4배) <br>
 리스크에 대한 언급이 많을 이후 재무지표 설명라인 혹은 어닝콜과 같이 언어적 요소로 구성되는 부분에 대해서 분석을 진행한다면 더욱 유의한 결과를 보일 것으로 기대된다.<br>
 추가적인 데이터 셋의 수집 등이 필요할 것으로 분석된다. <br>